In [15]:
import os
from typing import Tuple, List, Union
from collections import defaultdict


# Day 1

In [25]:

def first_digit_char(s: str) -> Tuple[int, int]:
    for i, c in enumerate(s):
        if c.isdigit():
            return i,int(c)
    raise Exception(f"No digit found in {s}")

def solve(lines) -> int:
    total_sum = 0
    for line in lines:
        number = int(f"{first_digit_char(line)[1]}{first_digit_char(reversed(line))[1]}")
        total_sum += number
    return total_sum

with open("input_day_1") as f:
    total_sum = solve(f)
        
print("Total sum is", total_sum)

Total sum is 55834


## Part 2

In [27]:
digits = ("one", "two", "three", "four", "five", "six", "seven", "eight", "nine")

def first_digit_word(s: str, digits: List[str]) -> Tuple[int,int]:
    min_pair = (len(s), None)
    for i,d in enumerate(digits):
        idx = s.find(d)
        if idx != -1 and idx < min_pair[0]:
            min_pair = (idx, i+1)
    return min_pair


def solve(lines) -> int:
    total_sum = 0
    for line in lines:
        _, d1 = min(first_digit_char(line),first_digit_word(line, digits), key=lambda t: t[0])

        line_reversed = line[::-1]
        digits_reversed = [d[::-1] for d in digits]
        _, d2 = min(first_digit_char(line_reversed),first_digit_word(line_reversed, digits_reversed), key=lambda t: t[0])

        number = int(f"{d1}{d2}")
        total_sum += number
    return total_sum

with open("input_day_1") as f:
    total_sum = solve(f)

print("Total Sum", total_sum)

Total Sum 53221


# Day 2

In [30]:


def game_string_to_dict(game_str: str) -> dict[str, int]:
    game, sets = game_str.split(": ")
    game_id = int(game.split(" ")[1])
    sets = sets.split("; ")

    game_stats = defaultdict(int)
    game_stats["Game_ID"] = game_id

    for set_string in sets:
        for color in set_string.split(", "):
            v,k = color.split(" ")
            assert k in ("red", "green", "blue"), f"not in colors '{k}'"
            game_stats[k] = max(int(v), game_stats[k])
    return game_stats

id_sum = 0
with open("input_day_2") as f:
    for line in f:
        line = line.rstrip("\n")
        game = game_string_to_dict(line)
        if game["red"] <= 12 and game["green"] <= 13 and game["blue"] <= 14:
            id_sum += game["Game_ID"]
print("Total sum of filtered Game IDs", id_sum)    

Total sum of filtered Game IDs 2348


In [33]:
from functools import reduce
from operator import mul


prod_sum = 0
with open("input_day_2") as f:
    for line in f:
        line = line.rstrip("\n")
        game = game_string_to_dict(line)
        cubes = [c for c in [game["red"], game["blue"], game["green"]] if c > 0]
        prod_sum += reduce(mul, cubes, 1)
print("Total sum of products of games", prod_sum)

Total sum of products of games 76008
